<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6a28f0e05f15bc21ac90942b19586bb8103c86525a6cb5cdeded94fcbf078ff8
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-02 09:03:37
-------------------
qualified stocks: 92
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.17 L
Current:  1.38 C
-------------------
Today PnL: 1.77 L (1.3%)
Current PnL: -18.88 L (-12.7%)
CY Booked + Current PnL: -9.84 L (-6.62%)
-------------------
Total profit:  3.19 L
Total loss:  -22.07 L
-------------------
Total Booked + Current PnL: 17.14 L (14.08%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.53%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 80.55 L (58.22%)
Deployed:  1.22 C
Current:  1.38 C
CAGR/XIRR %: 8.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-16.60,63.0,H-LC,7.95,115136.0,10824.0,5826.0,2.22,10.38,5.06,15.96,9.0,1.86,0.85,29.03,X5K,ATH,METALS
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.31,161936.0,35736.0,10445.0,1.60,28.32,6.45,36.59,108.0,3.42,1.19,44.43,XY24,NTT,HEALTHCARE
26,DIXON,18940.15,-29.68,68.0,H-MC,6.55,140656.0,25720.0,10859.0,5.34,22.38,7.72,31.83,81.0,2.37,1.03,45.79,X40N,ATH,IT
82,TTKPRESTIG,770.00,104.91,68.0,M-SC,7.65,88327.0,-12450.0,12542.0,2.66,-12.35,14.20,0.09,245.0,-0.99,0.65,16.04,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-15.52,55.0,H-LC,8.47,168773.0,12596.0,13502.0,0.81,8.07,8.00,16.71,22.0,0.93,1.24,26.91,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HAVELLS,2069.16,-0.94,60.0,H-MC,2.97,245575.0,-2156.0,77209.0,3.19,-0.87,31.44,30.30,92.0,-0.03,1.80,12.31,X40,ATH,ELECTRICAL
38,ICICIGI,2260.25,-20.81,35.0,H-MC,4.76,136182.0,49.0,31077.0,0.04,0.04,22.82,22.86,91.0,0.00,1.00,12.65,X40,ATH,INSURANCE
56,PGHH,18062.58,-31.21,51.0,H-MC,5.41,199275.0,-1545.0,71659.0,1.52,-0.77,35.96,34.92,80.0,-0.02,1.46,3.87,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RELIANCE,1533.00,-15.54,36.0,H-LC,7.97,211208.0,842.0,27943.0,-0.24,0.40,13.23,13.68,37.0,0.03,1.55,16.99,XY25,NTT,REFINERIES
33,HCLTECH,1943.91,1.08,42.0,H-LC,10.05,230539.0,-11385.0,74649.0,0.93,-4.71,32.38,26.15,8.0,-0.15,1.69,8.94,X40,ATH,IT
47,JIOFIN,387.00,-0.58,43.0,H-LC,14.96,219835.0,2429.0,51068.0,0.75,1.12,23.23,24.61,48.0,0.05,1.61,56.55,XY24,BTT,FINANCE
1,ABB,7934.00,-41.13,44.0,H-LC,13.81,246115.0,-15504.0,134723.0,2.57,-5.93,54.74,45.57,7.0,-0.12,1.81,4.72,AR,NTT,ELECTRICAL
45,ITC,452.00,-38.34,46.0,H-LC,3.11,198461.0,-1677.0,22565.0,-0.95,-0.84,11.37,10.44,4.0,-0.07,1.46,4.68,X40,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-5.78,53.0,H-LC,2.69,233865.0,-12286.0,91488.0,1.24,-4.99,39.12,32.18,16.0,-0.13,1.72,29.25,X200,ATH,IT
55,NESTLEIND,1377.0,-10.64,56.0,H-LC,3.09,274810.0,9384.0,47405.0,1.57,3.54,17.25,21.40,11.0,0.20,2.02,11.59,XY25,NTT,FMCG
45,ITC,452.0,-38.34,46.0,H-LC,3.11,198461.0,-1677.0,22565.0,-0.95,-0.84,11.37,10.44,4.0,-0.07,1.46,4.68,X40,NTT,FMCG
20,CIPLA,1795.0,-18.41,62.0,H-LC,4.71,217446.0,12946.0,28464.0,-0.14,6.33,13.09,20.25,10.0,0.45,1.60,15.62,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-10.45,65.0,H-LC,7.49,226178.0,-25590.0,147830.0,2.05,-10.16,65.36,48.55,27.0,-0.17,1.66,21.66,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VAIBHAVGBL,521.00,41.60,48.0,H-SC,11.92,94651.0,-57340.0,131461.0,6.77,-37.73,138.89,48.77,125.0,-0.44,0.70,10.17,XR,NTT,JEWELLERY
26,DIXON,18940.15,-29.68,68.0,H-MC,6.55,140656.0,25720.0,10859.0,5.34,22.38,7.72,31.83,81.0,2.37,1.03,45.79,X40N,ATH,IT
36,HINDZINC,730.22,16.37,55.0,M-LC,3.63,189616.0,-15460.0,128029.0,3.97,-7.54,67.52,54.89,52.0,-0.12,1.39,14.74,X5K,ATH,METALS
18,CAMS,5211.76,-5.38,49.0,H-SC,5.82,107514.0,5510.0,38415.0,3.63,5.40,35.73,43.06,122.0,0.14,0.79,23.44,X40N,ATH,MISC
14,BLUESTARCO,2326.38,12.85,66.0,H-MC,3.73,194830.0,30160.0,37817.0,3.54,18.32,19.41,41.28,89.0,0.80,1.43,27.76,X40N,ATH,AC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,66.0,L-SC,29.23,90662.0,-22887.0,62938.0,-5.62,-20.16,69.42,35.27,268.0,-0.36,0.67,121.90,XR,NTT,HOTELS
45,ITC,452.00,-38.34,46.0,H-LC,3.11,198461.0,-1677.0,22565.0,-0.95,-0.84,11.37,10.44,4.0,-0.07,1.46,4.68,X40,NTT,FMCG
37,HONAUT,58357.33,-18.32,52.0,H-SC,3.32,115155.0,-12783.0,59915.0,-0.92,-9.99,52.03,36.84,143.0,-0.21,0.85,17.89,X40N,ATH,ELECTRICAL
31,HATHWAY,31.40,910.87,40.0,H-SC,4.31,57960.0,-21000.0,67639.0,-0.82,-26.60,116.70,59.07,161.0,-0.31,0.43,16.29,XR,NTT,ENTERTAINMENT
8,AWL,485.00,-66.44,40.0,H-MC,6.80,206209.0,-62238.0,193919.0,-0.62,-23.18,94.04,49.05,116.0,-0.32,1.51,6.02,XY24,NTT,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,TTKPRESTIG,770.0,104.91,68.0,M-SC,7.65,88327.0,-12450.0,12542.0,2.66,-12.35,14.20,0.09,245.0,-0.99,0.65,16.04,OX40N,NTT,DURABLES
83,UJJIVANSFB,60.0,85.53,46.0,H-SC,14.36,116937.0,-25542.0,45068.0,1.48,-17.93,38.54,13.70,163.0,-0.57,0.86,38.02,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.52,52.0,H-SC,3.53,217467.0,-52200.0,88531.0,0.68,-19.36,40.71,13.47,138.0,-0.59,1.60,9.99,XY24,NTT,PAINTS
19,CERA,9475.0,-20.76,41.0,H-SC,3.06,113166.0,-30637.0,57386.0,-0.38,-21.30,50.71,18.60,149.0,-0.53,0.83,24.88,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.59,39.0,M-SC,2.69,136152.0,-38407.0,38436.0,-0.15,-22.00,28.23,0.02,221.0,-1.00,1.00,18.43,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,4998.00,-26.44,50.0,H-LC,15.05,292584.0,-53372.0,177218.0,0.90,-15.43,60.57,35.80,1.0,-0.30,2.15,3.65,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.56,319372.0,6484.0,165211.0,2.03,2.07,51.73,54.87,3.0,0.04,2.35,8.86,X40,BTT,IT
45,ITC,452.00,-38.34,46.0,H-LC,3.11,198461.0,-1677.0,22565.0,-0.95,-0.84,11.37,10.44,4.0,-0.07,1.46,4.68,X40,NTT,FMCG
88,VBL,671.64,-12.21,47.0,H-LC,7.76,316033.0,191.0,112476.0,1.68,0.06,35.59,35.67,5.0,0.00,2.32,13.92,X40N,ATH,FMCG
1,ABB,7934.00,-41.13,44.0,H-LC,13.81,246115.0,-15504.0,134723.0,2.57,-5.93,54.74,45.57,7.0,-0.12,1.81,4.72,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.31,161936.0,35736.0,10445.0,1.60,28.32,6.45,36.59,108.0,3.42,1.19,44.43,XY24,NTT,HEALTHCARE
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.80,183540.0,32627.0,24686.0,3.07,21.62,13.45,37.97,93.0,1.32,1.35,52.39,AR,ATH,AUTO
26,DIXON,18940.15,-29.68,68.0,H-MC,6.55,140656.0,25720.0,10859.0,5.34,22.38,7.72,31.83,81.0,2.37,1.03,45.79,X40N,ATH,IT
27,DMART,5201.00,-7.11,72.0,H-LC,13.40,232010.0,47529.0,22830.0,-0.44,25.76,9.84,38.14,38.0,2.08,1.70,39.10,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.45,59.0,H-MC,15.31,161936.0,35736.0,10445.0,1.60,28.32,6.45,36.59,108.0,3.42,1.19,44.43,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,12.85,66.0,H-MC,3.73,194830.0,30160.0,37817.0,3.54,18.32,19.41,41.28,89.0,0.80,1.43,27.76,X40N,ATH,AC
89,VOLTAS,1918.49,0.88,66.0,H-MC,5.91,211785.0,20043.0,75988.0,2.76,10.45,35.88,50.08,99.0,0.26,1.56,17.96,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.80,183540.0,32627.0,24686.0,3.07,21.62,13.45,37.97,93.0,1.32,1.35,52.39,AR,ATH,AUTO
18,CAMS,5211.76,-5.38,49.0,H-SC,5.82,107514.0,5510.0,38415.0,3.63,5.40,35.73,43.06,122.0,0.14,0.79,23.44,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.80,183540.0,32627.0,24686.0,3.07,21.62,13.45,37.97,93.0,1.32,1.35,52.39,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,52.0,H-SC,12.52,137811.0,14475.0,119289.0,0.12,11.74,86.56,108.45,119.0,0.12,1.01,36.71,AR,ATH,MISC
90,WHIRLPOOL,2270.00,-41.55,52.0,M-SC,5.31,98752.0,7254.0,71496.0,1.24,7.93,72.40,86.07,223.0,0.10,0.73,43.41,XR,NTT,DURABLES
54,NATIONALUM,247.44,-49.46,56.0,H-MC,1.34,101649.0,1145.0,29499.0,3.01,1.14,29.02,30.49,79.0,0.04,0.75,33.91,MH,ATH,METALS
91,WIPRO,420.00,-11.52,52.0,M-LC,7.58,155223.0,4278.0,105179.0,0.38,2.83,67.76,72.51,53.0,0.04,1.14,9.03,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-6.02,39.0,H-SC,16.47,87539.0,-13424.0,107953.0,1.51,-13.30,123.32,93.63,148.0,-0.12,0.64,30.39,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,66.0,L-SC,29.23,90662.0,-22887.0,62938.0,-5.62,-20.16,69.42,35.27,268.0,-0.36,0.67,121.90,XR,NTT,HOTELS
52,MASFIN,398.61,-18.25,51.0,H-SC,13.03,93015.0,-4965.0,26565.0,-0.02,-5.07,28.56,22.05,152.0,-0.19,0.68,35.20,XR,ATH,FINANCE
35,HINDALCO,756.01,-16.60,63.0,H-LC,7.95,115136.0,10824.0,5826.0,2.22,10.38,5.06,15.96,9.0,1.86,0.85,29.03,X5K,ATH,METALS
83,UJJIVANSFB,60.00,85.53,46.0,H-SC,14.36,116937.0,-25542.0,45068.0,1.48,-17.93,38.54,13.70,163.0,-0.57,0.86,38.02,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,66.0,L-SC,29.23,90662.0,-22887.0,62938.0,-5.62,-20.16,69.42,35.27,268.0,-0.36,0.67,121.90,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-1.09,75.0,H-MC,17.80,183540.0,32627.0,24686.0,3.07,21.62,13.45,37.97,93.0,1.32,1.35,52.39,AR,ATH,AUTO
14,BLUESTARCO,2326.38,12.85,66.0,H-MC,3.73,194830.0,30160.0,37817.0,3.54,18.32,19.41,41.28,89.0,0.80,1.43,27.76,X40N,ATH,AC
27,DMART,5201.00,-7.11,72.0,H-LC,13.40,232010.0,47529.0,22830.0,-0.44,25.76,9.84,38.14,38.0,2.08,1.70,39.10,X40N,NTT,FMCG
60,RELAXO,1176.00,-32.42,61.0,H-SC,7.84,62682.0,-52856.0,87849.0,2.53,-45.75,140.15,30.28,136.0,-0.60,0.46,23.23,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.40
1,25,42.62
2,50,72.15


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.75
SC    36.36
MC    25.90
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      15.26
X40N     14.06
XY25     11.18
AR        9.16
XR        8.80
OX40N     7.18
X5K       2.24
X200      1.72
MH        1.66
SR        1.05
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.71
H-MC    23.08
H-SC    21.86
M-SC    12.95
M-LC     7.01
M-MC     2.50
L-SC     1.55
L-LC     1.03
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.87,-0.81,33.65
IT,13.15,-15.26,72.43
BANKS,7.16,-16.92,66.57
FINANCE,7.10,-24.46,70.35
PAINTS,5.89,-15.22,41.01
MISC,5.76,-17.13,84.27
ELECTRICAL,5.61,-7.80,67.22
AUTO,4.53,-9.65,57.36
AC,3.70,3.86,28.71


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2863112.0,23
AR,1231312.0,10
XR,1096930.0,13
X40,889747.0,10
X40N,518647.0,11
OX40N,481171.0,10
XY25,459530.0,8
SR,199666.0,2
X5K,133855.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2796711.0,27
M-SC,1482662.0,17
H-LC,1425942.0,18
H-MC,1233780.0,18
M-LC,423976.0,5
M-MC,336489.0,2
L-SC,251613.0,3
L-MC,59686.0,1
L-LC,44422.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,944429.0,6
M-SC,XY24,868465.0,7
H-SC,AR,669155.0,5
H-LC,X40,587473.0,5
H-SC,XR,529539.0,6
H-MC,XY24,486995.0,5
H-LC,AR,376277.0,2
M-MC,XY24,336489.0,2
H-MC,X40,220891.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
